<a href="https://colab.research.google.com/github/bbcx-investments/python/blob/main/option_portfolios.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import plotly.express as px

class Option(list):
    def add(self,security,quantity,strike=None) :
        self.append(dict(security=security,quantity=quantity,strike=strike))
    def strikes(self) :
        return [d['strike'] for d in self if d['security']!='Underlying']
    def grid(self) :
        strikes = self.strikes()
        maxstrike = 100 if len(strikes)==0 else 1.5*np.max(strikes)
        return np.linspace(0,maxstrike,200)
    def value(self) :
        grid = self.grid()
        value = np.zeros(len(grid))
        for x in self :
            if x['security']=='Underlying' :
                value += x['quantity']*grid
            elif x['security'] == 'Call' :
                value += x['quantity']*np.maximum(grid-x['strike'],0)
            else :
                value += x['quantity']*np.maximum(x['strike']-grid,0)
        return value
    def plot(self) :
        df = pd.DataFrame(self.grid(),columns=['Underlying'])
        df['Portfolio'] = self.value()
        fig = px.line(df,x='Underlying',y='Portfolio')
        fig.layout.xaxis['title'] = 'Underlying Price'
        fig.layout.yaxis['title'] = 'Portfolio Value'
        fig.update_layout(margin=dict(l=60, r=20, t=20, b=60)) 
        fig.update_xaxes(title_font_size=16) 
        fig.update_yaxes(title_font_size=16) 
        fig.update_layout(font_size=14)
        fig.layout.template='plotly_dark'
        fig.update_layout(xaxis_tickprefix = '$', xaxis_tickformat = ',.0f')
        fig.update_layout(yaxis_tickprefix = '$', yaxis_tickformat = ',.0f')
        return fig
    
# Example

x = Option()
x.add(security='Underlying',quantity=1)
x.add(security='Call',strike=40,quantity=-1)
x.plot()